In [1]:
# install neo4j-driver
!pip install neo4j-driver


In [2]:
from neo4j.v1 import GraphDatabase, basic_auth
import requests
import json

# Loading the OSCON Graph

The OSCON schedule data is available for download [here](http://conferences.oreilly.com/oscon/open-source-us/public/content/report/schedule_feed). We can model this data as a graph using the data model below. This Python script shows how we can import the data into Neo4j.

![](https://raw.githubusercontent.com/neo4j-examples/conf-graph/master/img/conf-graph.jpg)

In [3]:
OSCON_FEED_URL = "http://conferences.oreilly.com/oscon/open-source-us/public/content/report/schedule_feed"

In [4]:
r = requests.get(OSCON_FEED_URL)
d = r.json()

In [5]:
d['Schedule'].keys()

dict_keys(['speakers', 'events', 'venues', 'conferences'])

In [6]:
INSERT_EVENTS_QUERY = '''
WITH {events} AS events
UNWIND events AS event
MERGE (t:Talk {serial: event.serial})
  ON CREATE SET t.name = event.name,
    t.type = event.event_type,
    t.time_start = event.time_start,
    t.time_stop = event.time_stop,
    t.description = event.description,
    t.url = event.website_url,
    t.image = event.large_img_url,
    t.youtube_url = event.youtube_url

MERGE (r:Room {serial: event.venue_serial})
CREATE UNIQUE (t)-[:IN]->(r)

FOREACH (speaker IN event.speakers |
  MERGE (s:Speaker {serial: speaker})
  CREATE UNIQUE (s)-[:PRESENTS]->(t)
)

FOREACH (cat in event.categories |
  MERGE (top:Topic {name: cat})
  CREATE UNIQUE (t)-[:ABOUT]->(top)
  
  MERGE (trac:Track {name: cat})
  CREATE UNIQUE (t)-[:PART_OF]->(trac)
) 
'''

INSERT_SPEAKERS_QUERY = '''
WITH {speakers} AS speakers
UNWIND speakers AS speaker
MERGE (s:Speaker {serial: speaker.serial})
  SET s.name = speaker.name,
    s.photo = speaker.photo,
    s.url = speaker.url,
    s.position = speaker.position,
    s.twitter = speaker.twitter,
    s.bio = speaker.bio,
    s.image = speaker.large_img_url,
    s.youtube = speaker.youtube_url

WITH s,speaker WHERE speaker.affiliation IS NOT NULL
MERGE (org:Organization {name: speaker.affiliation})
CREATE UNIQUE (s)-[:AFFILIATED]->(org)
'''

INSERT_VENUES = '''
WITH {venues} AS venues
UNWIND venues AS venue
MERGE (r:Room {serial: venue.serial})
SET r.name = venue.name

'''

In [7]:
# instantiate the driver and create a session
driver = GraphDatabase.driver("bolt://localhost", auth=basic_auth("neo4j", "letmein"))
session = driver.session()

In [8]:
session.run(INSERT_EVENTS_QUERY, parameters={'events': d['Schedule']['events']})

In [9]:
session.run(INSERT_SPEAKERS_QUERY, parameters={'speakers': d['Schedule']['speakers']})

In [10]:
session.run(INSERT_VENUES,parameters={'venues': d['Schedule']['venues']})

In [11]:
# What organizations in "The New Stuff" track have the most speakers?
result = session.run('''
    MATCH (track:Track {name: "The New Stuff"}) 
    MATCH (track)<-[:PART_OF]-(t:Talk)<-[:PRESENTS]-(s:Speaker)
    MATCH (s)-[:AFFILIATED]->(org:Organization)
    RETURN org.name AS organization, count(*) AS num 
    ORDER BY num DESC LIMIT 10
''')

In [12]:
for record in result:
    print("%s: %s" % (record['organization'],record['num']))

Microsoft: 4
Secret Lab Pty. Ltd.: 4
lonely.coffee: 2
Ardan Labs: 2
University of Cambridge, UK: 1
Docker, Inc: 1
WNYC Radio: 1
Etsy: 1
Docker: 1
Dominant Paradigm Subversion, Inc: 1
